Install all the required libraries

In [ ]:
# !pip install -q google-generativeai
import pprint
import google.generativeai as genai
import os
import json
import pandas as pd
import requests
from bs4 import BeautifulSoup
from google.colab import userdata
import re

--------------------------------------------------------------------------------

Add the Gemini model API key to the environment

In [ ]:
os.environ['GOOGLE_API_KEY'] = ""

genai.configure(api_key=os.environ['GOOGLE_API_KEY'])

--------------------------------------------------------------------------------

Load the model

In [ ]:
model = genai.GenerativeModel('gemini-pro')
model

 genai.GenerativeModel(
   model_name='models/gemini-pro',
   generation_config={}.
   safety_settings={}
)

--------------------------------------------------------------------------------

Helper function to fetch raw text from a webpage url

In [ ]:
def get_raw_text(url):
  response = requests.get(url,headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'})
  soup = BeautifulSoup(response.content, "html.parser")
  for tag in soup(["script", "style", "head", "header", "footer"]):
    tag.extract()
  text = soup.text
  lines = (line.strip() for line in text.splitlines())
  chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
  return '\n'.join(chunk for chunk in chunks if chunk)

--------------------------------------------------------------------------------

Setting the base url of IMDB website to get access to top 250 movies

In [ ]:
url = 'https://www.imdb.com/chart/top/'

--------------------------------------------------------------------------------

Helper function that takes the base url and return a list of urls for the top 250 movies

In [ ]:
def get_movie_list(base_url):
  root_url = url.rstrip('chart/top/')
  response = requests.get(base_url,headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'})
  soup = BeautifulSoup(response.text, 'html.parser')
  movies = soup.find('ul', class_="ipc-metadata-list--base").find_all('li')
  # print(movies)
  movie_links = []
  for movie in movies:
    anchor_tag = movie.find('a')
    link = anchor_tag.get('href')
    link = root_url+link
    movie_links.append(link)
  return movie_links

Calling the above function to fetch all the 250 urls for the top rated movies

In [ ]:
movie_links = get_movie_list(url)
len(movie_links)

250

Accessing the first 10 movie links from the above list

In [ ]:
movie_links[:10]

['https://www.imdb.com/title/tt0111161/?ref_=chttp_i_1',
 'https://www.imdb.com/title/tt0068646/?ref_=chttp_i_2',
 'https://www.imdb.com/title/tt0468569/?ref_=chttp_i_3',
 'https://www.imdb.com/title/tt0071562/?ref_=chttp_i_4',
 'https://www.imdb.com/title/tt0050083/?ref_=chttp_i_5',
 'https://www.imdb.com/title/tt0108052/?ref_=chttp_i_6',
 'https://www.imdb.com/title/tt0167260/?ref_=chttp_i_7',
 'https://www.imdb.com/title/tt0110912/?ref_=chttp_i_8',
 'https://www.imdb.com/title/tt0120737/?ref_=chttp_i_9',
 'https://www.imdb.com/title/tt0060196/?ref_=chttp_i_10']

--------------------------------------------------------------------------------

Setting up the prompt to fetch all the necessary details for each movies.

In [ ]:
def get_prompt(content):
  # An example with necessary field that were manually extracted to perform One-shot prompting
  example = {
    "Movie_Name": "The Shawshank Redemption",
    "Release_Yr": "1994",
    "Movie_Rating": "R",
    "Duration": "2h 22m",
    "IMDb_Rating": "9.3/10",
    "Popularity": "745",
    "Genre": "Drama",
    "Movie_Desc": "Over the course of several years, two convicts form a friendship, seeking consolation and, eventually, redemption through basic compassion.",
    "Director": "Frank Darabont",
    "Writers": ["Stephen King", "Frank Darabont"],
    "Movie_stars": ["Tim Robbins", "Morgan Freeman", "Bob Gunton"],
    "Oscars_n": "7"
  }


  prompt = f"""
  Extract movie information from the text provided below. Carefully identify the following items:

  - Movie name
  - Release year
  - Parental guidance rating (considering language, violence, etc.)
  - Duration
  - IMDb rating
  - Popularity
  - Genre
  - Movie description
  - Director
  - Writers
  - Stars
  - Number of Oscar nominations

  Return a JSON object with the following fields:
  - Movie_Name,
  - Release_Yr,
  - Movie_Rating,
  - Duration,
  - IMDb_Rating,
  - Popularity,
  - Genre,
  - Movie_Desc,
  - Director,
  - Writers,
  - Movie_stars,
  - Oscars_n
  If the information isn't present, use "NA" as the value.

  Below is an example JSON object with all keys correctly extracted:
  ---
  Example: {example}
  ---

  Text: '''{content}'''
  """

  return prompt

--------------------------------------------------------------------------------

Helper function to convert the response from LLM to a valid JSON object

In [ ]:
def extract_json(text_response):
    # This pattern matches a string that starts with '{' and ends with '}'
    pattern = r'\{[^{}]*\}'
    matches = re.finditer(pattern, text_response)
    json_objects = []
    for match in matches:
        json_str = match.group(0)
        try:
            # Validate if the extracted string is valid JSON
            json_obj = json.loads(json_str)
            json_objects.append(json_obj)
        except json.JSONDecodeError:
            # Extend the search for nested structures
            extended_json_str = extend_search(text_response, match.span())
            try:
                json_obj = json.loads(extended_json_str)
                # json_objects.append(json_obj)
            except json.JSONDecodeError:
                # Handle cases where the extraction is not valid JSON
                continue
    if json_obj:
        return json_obj
    else:
        return None  # Or handle this case as you prefer
def extend_search(text, span):
    # Extend the search to try to capture nested structures
    start, end = span
    nest_count = 0
    for i in range(start, len(text)):
        if text[i] == '{':
            nest_count += 1
        elif text[i] == '}':
            nest_count -= 1
            if nest_count == 0:
                return text[start:i+1]
    return text[start:end]

Extracting data for all the top 250 movies using the above prompt on Gemini model

In [ ]:
import time

def extraction_using_LLM(movie_links):
    model = genai.GenerativeModel('gemini-pro')
    final_json = []
    for link in movie_links:
        try:
            raw_text = get_raw_text(link)
            # print(raw_text)
            prompt = get_prompt(raw_text)
            response = model.generate_content(prompt)
            json_objects = extract_json(response.text)
            print(f'Data successfully scraped for - {link}')
            final_json.append(json_objects)
            # print(final_json)
        except Exception as e:
            print(f"Error occurred while processing link: {link}. Error message: {str(e)}")
            final_json.append({})  # Append an empty dictionary if an error occurs
        finally:
            time.sleep(2)  # Wait for 2 seconds before processing the next link
    return final_json


In [ ]:
json_data = extraction_using_LLM(movie_links)

Data successfully scraped for - https://www.imdb.com/title/tt0111161/?ref_=chttp_i_1
Data successfully scraped for - https://www.imdb.com/title/tt0068646/?ref_=chttp_i_2
Data successfully scraped for - https://www.imdb.com/title/tt0468569/?ref_=chttp_i_3
Error occurred while processing link: https://www.imdb.com/title/tt0071562/?ref_=chttp_i_4. Error message: local variable 'json_obj' referenced before assignment
Data successfully scraped for - https://www.imdb.com/title/tt0050083/?ref_=chttp_i_5
Data successfully scraped for - https://www.imdb.com/title/tt0108052/?ref_=chttp_i_6
Data successfully scraped for - https://www.imdb.com/title/tt0167260/?ref_=chttp_i_7
Data successfully scraped for - https://www.imdb.com/title/tt0110912/?ref_=chttp_i_8
Data successfully scraped for - https://www.imdb.com/title/tt0120737/?ref_=chttp_i_9
Data successfully scraped for - https://www.imdb.com/title/tt0060196/?ref_=chttp_i_10
Data successfully scraped for - https://www.imdb.com/title/tt0109830/?re

In [ ]:
json_data

In [ ]:
df = pd.DataFrame(json_data)
df

,Movie_Name,Release_Yr,Movie_Rating,Duration,IMDb_Rating,Popularity,Genre,Movie_Desc,Director,Writers,Movie_stars,Oscars_n,Movie_Stars
0,The Shawshank Redemption,1994,R,2h 22m,9.3/10,722,Drama,"Over the course of several years, two convicts...",Frank Darabont,"[Stephen King, Frank Darabont]","[Tim Robbins, Morgan Freeman, Bob Gunton]",7,NaN
1,The Godfather,1972,R,2h 55m,9.2/10,805,CrimeDrama,The aging patriarch of an organized crime dyna...,Francis Ford Coppola,"[Mario Puzo, Francis Ford Coppola]","[Marlon Brando, Al Pacino, James Caan, Diane K...",NA,NaN
2,The Dark Knight,2008,PG-13,2h 32m,9.0/10,10810,"Action, Crime, Drama",When the menace known as the Joker wreaks havo...,Christopher Nolan,"[Jonathan Nolan, Christopher Nolan, David S. G...","[Christian Bale, Heath Ledger, Aaron Eckhart, ...",2,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,12 Angry Men,1957,Approved,1h 36m,9.0/10,33931,"Crime, Drama",The jury in a New York City murder trial is fr...,Sidney Lumet,[Reginald Rose],"[Henry Fonda, Lee J. Cobb, Martin Balsam, John...",3,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,The Help,2011,PG-13,2h 26m,8.1/10,61495,Drama,An aspiring author during the civil rights mov...,Tate Taylor,"[Tate Taylor, Kathryn Stockett]","[Viola Davis, Emma Stone, Octavia Spencer]",1,NaN
246,It Happened One Night,1934,Passed,1h 45m,8.1/10,"4,030",ComedyRomance,A renegade reporter trailing a young runaway h...,Frank Capra,"[Robert Riskin, Samuel Hopkins Adams]","[Clark Gable, Claudette Colbert, Walter Connolly]",5,NaN
247,Aladdin,1992,G,1h 30m,8.0/10,"1,216","Animation, Adventure, Comedy",A kind-hearted street urchin and a power-hungr...,"Ron Clements, John Musker","[Ron Clements, John Musker, Ted Elliott]","[Scott Weinger, Robin Williams, Linda Larkin, ...",2,NaN
248,Dances with Wolves,1990,PG-13,3h 1m,8.0/10,"1,108,405","Adventure, Drama, Western","Lieutenant John Dunbar, assigned to a remote w...",Kevin Costner,[Michael Blake],"[Kevin Costner, Mary McDonnell, Graham Greene]",7,NaN


In [ ]:
df.to_csv('/content/drive/MyDrive/Independent_Study/Movies_data.csv')

In [1]:
import pandas as pd

In [2]:
movies_data = pd.read_csv('/content/drive/MyDrive/Independent_Study/Movies_data.csv')
movies_data.shape

(250, 14)

In [3]:
movies_data.columns

Index(['Unnamed: 0', 'Movie_Name', 'Release_Yr', 'Movie_Rating', 'Duration',
       'IMDb_Rating', 'Popularity', 'Genre', 'Movie_Desc', 'Director',
       'Writers', 'Movie_stars', 'Oscars_n', 'Movie_Stars'],
      dtype='object')

In [4]:
movies_data = movies_data.drop(columns=['Unnamed: 0', 'Movie_Stars'])
movies_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Movie_Name    213 non-null    object
 1   Release_Yr    213 non-null    object
 2   Movie_Rating  205 non-null    object
 3   Duration      213 non-null    object
 4   IMDb_Rating   213 non-null    object
 5   Popularity    208 non-null    object
 6   Genre         213 non-null    object
 7   Movie_Desc    213 non-null    object
 8   Director      213 non-null    object
 9   Writers       213 non-null    object
 10  Movie_stars   212 non-null    object
 11  Oscars_n      181 non-null    object
dtypes: object(12)
memory usage: 23.6+ KB


In [5]:
movies_data.head()

,Movie_Name,Release_Yr,Movie_Rating,Duration,IMDb_Rating,Popularity,Genre,Movie_Desc,Director,Writers,Movie_stars,Oscars_n
0,The Shawshank Redemption,1994,R,2h 22m,9.3/10,722,Drama,"Over the course of several years, two convicts...",Frank Darabont,"['Stephen King', 'Frank Darabont']","['Tim Robbins', 'Morgan Freeman', 'Bob Gunton']",7
1,The Godfather,1972,R,2h 55m,9.2/10,805,CrimeDrama,The aging patriarch of an organized crime dyna...,Francis Ford Coppola,"['Mario Puzo', 'Francis Ford Coppola']","['Marlon Brando', 'Al Pacino', 'James Caan', '...",NaN
2,The Dark Knight,2008,PG-13,2h 32m,9.0/10,10810,"Action, Crime, Drama",When the menace known as the Joker wreaks havo...,Christopher Nolan,"['Jonathan Nolan', 'Christopher Nolan', 'David...","['Christian Bale', 'Heath Ledger', 'Aaron Eckh...",2
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,12 Angry Men,1957,Approved,1h 36m,9.0/10,33931,"Crime, Drama",The jury in a New York City murder trial is fr...,Sidney Lumet,['Reginald Rose'],"['Henry Fonda', 'Lee J. Cobb', 'Martin Balsam'...",3


Finding all the rows in the dataframe whhere all of its columns are null. We will remove these rows as it doesn't make any significant impact on our analysis

In [6]:
num_empty_rows = movies_data.isnull().all(axis=1).sum()

print("Number of rows in Movies dataframe where all the columns are empty:", num_empty_rows)

Number of rows in Movies dataframe where all the columns are empty: 37


In [7]:
movies_data_cleaned = movies_data.dropna(how='all')
movies_data_cleaned.shape

(213, 12)

In [8]:
movies_data_cleaned.head()

,Movie_Name,Release_Yr,Movie_Rating,Duration,IMDb_Rating,Popularity,Genre,Movie_Desc,Director,Writers,Movie_stars,Oscars_n
0,The Shawshank Redemption,1994,R,2h 22m,9.3/10,722,Drama,"Over the course of several years, two convicts...",Frank Darabont,"['Stephen King', 'Frank Darabont']","['Tim Robbins', 'Morgan Freeman', 'Bob Gunton']",7
1,The Godfather,1972,R,2h 55m,9.2/10,805,CrimeDrama,The aging patriarch of an organized crime dyna...,Francis Ford Coppola,"['Mario Puzo', 'Francis Ford Coppola']","['Marlon Brando', 'Al Pacino', 'James Caan', '...",NaN
2,The Dark Knight,2008,PG-13,2h 32m,9.0/10,10810,"Action, Crime, Drama",When the menace known as the Joker wreaks havo...,Christopher Nolan,"['Jonathan Nolan', 'Christopher Nolan', 'David...","['Christian Bale', 'Heath Ledger', 'Aaron Eckh...",2
4,12 Angry Men,1957,Approved,1h 36m,9.0/10,33931,"Crime, Drama",The jury in a New York City murder trial is fr...,Sidney Lumet,['Reginald Rose'],"['Henry Fonda', 'Lee J. Cobb', 'Martin Balsam'...",3
5,Schindler's List,1993,R,3h 15m,9.0/10,1829,"Biography, Drama, History","In German-occupied Poland during World War II,...",Steven Spielberg,"['Thomas Keneally', 'Steven Zaillian']","['Liam Neeson', 'Ralph Fiennes', 'Ben Kingsley...",7


In [9]:
movies_data_cleaned['IMDb_Rating'].unique()

array(['9.3/10', '9.2/10', '9.0/10', '8.9/10\n2.2M', '8.9/10', '8.8/10',
       '8.7/10', '8.6/10', '8.5/10', '8.4/10', '8.3/101.2', '8.3/10',
       '8.2/10', '8.1/10', '8.0/10'], dtype=object)

In [12]:
movies_data_cleaned['IMDb_Rating'] = movies_data_cleaned['IMDb_Rating'].str.split('/').str[0]

movies_data_cleaned['IMDb_Rating'] = pd.to_numeric(movies_data_cleaned['IMDb_Rating'])

movies_data_cleaned['IMDb_Rating'].unique()

<ipython-input-12-09e628cc2f80>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_data_cleaned['IMDb_Rating'] = movies_data_cleaned['IMDb_Rating'].str.split('/').str[0]
<ipython-input-12-09e628cc2f80>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_data_cleaned['IMDb_Rating'] = pd.to_numeric(movies_data_cleaned['IMDb_Rating'])


array([9.3, 9.2, 9. , 8.9, 8.8, 8.7, 8.6, 8.5, 8.4, 8.3, 8.2, 8.1, 8. ])

In [14]:
movies_data_cleaned['Release_Yr'].unique()

array(['1994', '1972', '2008', '1957', '1993', '2003', '2001', '1966',
       '2002', '1999', '2010', '1980', '2024', '1990', '1975', '1995',
       '2014', '1946', '1954', '1991', '1998', '1997', '1977', '2019',
       '1960', 'June 2, 2023 (United States)', '2000', '2006', '1988',
       '1962', '1942', '2011', '1936', '1968', '1979', '1931', '1981',
       '1950', '2018', '1940', '1986', '2009', '1964', '1984', '2016',
       '1952', '1985', '2023', '1983', '2004', '2012', '1959', '1958',
       '1944', '1987', '1971', '1973', '1989', '1927', '2007', '2020',
       '1948', '1965', '2005', '1961', '1992', '1982', '1963', '2013',
       '1974', '1939', '2015', '1996', '1925', '1926', '1978', '1924',
       '1949', '1953', '1976', '2017', '1928', '2021', '1967', '1934'],
      dtype=object)

In [21]:
movies_data_cleaned['Popularity'].unique()

array(['722', '805', '10810', '33931', '1829', '27647', '1221', '13412',
       '4702', '21017', '54087', '16013', '13223', '80043', '31', '10191',
       '1.3M', '38767', '23070', '6711', '1,65855', '1,417254', '1944',
       '1.5M', '77746', '32560', '41319', '34315', '5968', '1791',
       '82314', '16221', '1495', '41233', '25616', '24472', '44421',
       '22816', '21626', '1,268', '1,771101', '38595', '716180', '82894',
       '1,726151', '259', '1,545107', '937365', '17238', nan, '111242',
       '35928', '894167', '67970', '1,23613', '2,096', '2,990', '66294',
       '3,273506', '40017', '4,534', '24846', '19918', '94082', '27112',
       '34624', '495217', '77236', '2,054158', '25315', '48622', '14825',
       '1,211166', '1874', '82046', '32230', '1,11874', '1,782', '1,653',
       '29026', '465355', '121', '1,370171', '1,00781', '25172', '68262',
       '43620', '3,920277', '935111', '1,534787', '45293', '1,613,197',
       '2,539', '1,682', '1,07360', '626147', '26345', '43

In [17]:
import re

def tranform_release_Yr(date_str):
  try:
    return int(date_str)
  except:
    match = re.search(r'\b\d{4}\b', date_str)
    if match:
        return int(match.group(0))
    else:
        return None

In [22]:
movies_data_cleaned['Release_Yr'] = movies_data_cleaned['Release_Yr'].apply(tranform_release_Yr)

movies_data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 213 entries, 0 to 249
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Movie_Name    213 non-null    object 
 1   Release_Yr    213 non-null    int64  
 2   Movie_Rating  205 non-null    object 
 3   Duration      213 non-null    object 
 4   IMDb_Rating   213 non-null    float64
 5   Popularity    208 non-null    object 
 6   Genre         213 non-null    object 
 7   Movie_Desc    213 non-null    object 
 8   Director      213 non-null    object 
 9   Writers       213 non-null    object 
 10  Movie_stars   212 non-null    object 
 11  Oscars_n      181 non-null    object 
dtypes: float64(1), int64(1), object(10)
memory usage: 21.6+ KB


In [24]:
movies_data_cleaned['Popularity'].fillna('0', inplace=True)

<ipython-input-24-26415f49dc83>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_data_cleaned['Popularity'].fillna('0', inplace=True)


Replace all the NaNs in the column 'Number of Oscars' by 0 and change the datatype to integer

In [25]:
movies_data_cleaned['Oscars_n'].unique()

array(['7', nan, '2', '3', '11', '1', '4', '2 wins & 5 nominations', '6',
       '5', '5 wins & 16 nominations', '4 wins & 1 nomination',
       '6 wins & 9 nominations', '8', '15', '27 wins & 18 nominations',
       'Won 1 Oscar\n99 wins & 118 nominations total',
       '1 win & 7 nominations', '2 wins & 1 nomination',
       '9 wins & 2 nominations'], dtype=object)

In [29]:
def classify_oscar(row):
  if pd.isna(row['Oscars_n']):
        return (0, 0, 0)
  else:
    # Regular expression pattern to match the strings
    pattern = r'(?:(?P<oscars>\d+) win(?:s)? & )?(?P<nominations>\d+) nomination(?:s)?'
    match = re.match(pattern, row['Oscars_n'])
    if match:
        oscars = int(match.group('oscars')) if match.group('oscars') else 0
        nominations = int(match.group('nominations'))
        return (oscars, 0, nominations)

    pattern = r'(?:(?P<oscars>\d+) win(?:s)? & )?(?P<wins>\d+) wins & (?P<nominations>\d+) nominations?'
    match = re.match(pattern, row['Oscars_n'])
    if match:
        oscars = int(match.group('oscars')) if match.group('oscars') else 0
        wins = int(match.group('wins'))
        nominations = int(match.group('nominations'))
        return (oscars, wins, nominations)

    pattern = r'Won (?P<oscar_won>\d+) Oscar\n(?P<wins>\d+) wins & (?P<nominations>\d+) nominations total'
    match = re.match(pattern, row['Oscars_n'])
    if match:
        oscar_won = int(match.group('oscar_won'))
        wins = int(match.group('wins'))
        nominations = int(match.group('nominations'))
        return (oscar_won, wins, nominations)

    # If none of the patterns match, return the value as Oscars won
    return (int(row['Oscars_n']), 0, 0)

# Apply the function to create new columns
movies_data_cleaned[['Oscars_Won', 'Awards_Won', 'Nominations']] = movies_data_cleaned.apply(classify_oscar, axis=1, result_type='expand')

<ipython-input-29-e892140e0952>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_data_cleaned[['Oscars_Won', 'Awards_Won', 'Nominations']] = movies_data_cleaned.apply(classify_oscar, axis=1, result_type='expand')
<ipython-input-29-e892140e0952>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_data_cleaned[['Oscars_Won', 'Awards_Won', 'Nominations']] = movies_data_cleaned.apply(classify_oscar, axis=1, result_type='expand')
<ipython-input-29-e892140e0952>:33: SettingWithCopyWarning: 
A value i

In [30]:
movies_data_cleaned.head()

,Movie_Name,Release_Yr,Movie_Rating,Duration,IMDb_Rating,Popularity,Genre,Movie_Desc,Director,Writers,Movie_stars,Oscars_n,Oscars_Won,Awards_Won,Nominations
0,The Shawshank Redemption,1994,R,2h 22m,9.3,722,Drama,"Over the course of several years, two convicts...",Frank Darabont,"['Stephen King', 'Frank Darabont']","['Tim Robbins', 'Morgan Freeman', 'Bob Gunton']",7,7,0,0
1,The Godfather,1972,R,2h 55m,9.2,805,CrimeDrama,The aging patriarch of an organized crime dyna...,Francis Ford Coppola,"['Mario Puzo', 'Francis Ford Coppola']","['Marlon Brando', 'Al Pacino', 'James Caan', '...",NaN,0,0,0
2,The Dark Knight,2008,PG-13,2h 32m,9.0,10810,"Action, Crime, Drama",When the menace known as the Joker wreaks havo...,Christopher Nolan,"['Jonathan Nolan', 'Christopher Nolan', 'David...","['Christian Bale', 'Heath Ledger', 'Aaron Eckh...",2,2,0,0
4,12 Angry Men,1957,Approved,1h 36m,9.0,33931,"Crime, Drama",The jury in a New York City murder trial is fr...,Sidney Lumet,['Reginald Rose'],"['Henry Fonda', 'Lee J. Cobb', 'Martin Balsam'...",3,3,0,0
5,Schindler's List,1993,R,3h 15m,9.0,1829,"Biography, Drama, History","In German-occupied Poland during World War II,...",Steven Spielberg,"['Thomas Keneally', 'Steven Zaillian']","['Liam Neeson', 'Ralph Fiennes', 'Ben Kingsley...",7,7,0,0


In [31]:
movies_data_cleaned = movies_data_cleaned.drop(columns=['Oscars_n'])
movies_data_cleaned.head()

,Movie_Name,Release_Yr,Movie_Rating,Duration,IMDb_Rating,Popularity,Genre,Movie_Desc,Director,Writers,Movie_stars,Oscars_Won,Awards_Won,Nominations
0,The Shawshank Redemption,1994,R,2h 22m,9.3,722,Drama,"Over the course of several years, two convicts...",Frank Darabont,"['Stephen King', 'Frank Darabont']","['Tim Robbins', 'Morgan Freeman', 'Bob Gunton']",7,0,0
1,The Godfather,1972,R,2h 55m,9.2,805,CrimeDrama,The aging patriarch of an organized crime dyna...,Francis Ford Coppola,"['Mario Puzo', 'Francis Ford Coppola']","['Marlon Brando', 'Al Pacino', 'James Caan', '...",0,0,0
2,The Dark Knight,2008,PG-13,2h 32m,9.0,10810,"Action, Crime, Drama",When the menace known as the Joker wreaks havo...,Christopher Nolan,"['Jonathan Nolan', 'Christopher Nolan', 'David...","['Christian Bale', 'Heath Ledger', 'Aaron Eckh...",2,0,0
4,12 Angry Men,1957,Approved,1h 36m,9.0,33931,"Crime, Drama",The jury in a New York City murder trial is fr...,Sidney Lumet,['Reginald Rose'],"['Henry Fonda', 'Lee J. Cobb', 'Martin Balsam'...",3,0,0
5,Schindler's List,1993,R,3h 15m,9.0,1829,"Biography, Drama, History","In German-occupied Poland during World War II,...",Steven Spielberg,"['Thomas Keneally', 'Steven Zaillian']","['Liam Neeson', 'Ralph Fiennes', 'Ben Kingsley...",7,0,0


In [32]:
movies_data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 213 entries, 0 to 249
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Movie_Name    213 non-null    object 
 1   Release_Yr    213 non-null    int64  
 2   Movie_Rating  205 non-null    object 
 3   Duration      213 non-null    object 
 4   IMDb_Rating   213 non-null    float64
 5   Popularity    213 non-null    object 
 6   Genre         213 non-null    object 
 7   Movie_Desc    213 non-null    object 
 8   Director      213 non-null    object 
 9   Writers       213 non-null    object 
 10  Movie_stars   212 non-null    object 
 11  Oscars_Won    213 non-null    int64  
 12  Awards_Won    213 non-null    int64  
 13  Nominations   213 non-null    int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 25.0+ KB


In [33]:
movies_data_cleaned.to_csv('/content/drive/MyDrive/Independent_Study/Final_data.csv')

In [2]:
import pandas as pd

data = pd.read_csv('/content/drive/MyDrive/Independent_Study/Final_data.csv')

In [4]:
data = data.drop(columns=['Unnamed: 0'])
data.head()

,Movie_Name,Release_Yr,Movie_Rating,Duration,IMDb_Rating,Popularity,Genre,Movie_Desc,Director,Writers,Movie_stars,Oscars_Won,Awards_Won,Nominations
0,The Shawshank Redemption,1994,R,2h 22m,9.3,722,Drama,"Over the course of several years, two convicts...",Frank Darabont,"['Stephen King', 'Frank Darabont']","['Tim Robbins', 'Morgan Freeman', 'Bob Gunton']",7,0,0
1,The Godfather,1972,R,2h 55m,9.2,805,CrimeDrama,The aging patriarch of an organized crime dyna...,Francis Ford Coppola,"['Mario Puzo', 'Francis Ford Coppola']","['Marlon Brando', 'Al Pacino', 'James Caan', '...",0,0,0
2,The Dark Knight,2008,PG-13,2h 32m,9.0,10810,"Action, Crime, Drama",When the menace known as the Joker wreaks havo...,Christopher Nolan,"['Jonathan Nolan', 'Christopher Nolan', 'David...","['Christian Bale', 'Heath Ledger', 'Aaron Eckh...",2,0,0
3,12 Angry Men,1957,Approved,1h 36m,9.0,33931,"Crime, Drama",The jury in a New York City murder trial is fr...,Sidney Lumet,['Reginald Rose'],"['Henry Fonda', 'Lee J. Cobb', 'Martin Balsam'...",3,0,0
4,Schindler's List,1993,R,3h 15m,9.0,1829,"Biography, Drama, History","In German-occupied Poland during World War II,...",Steven Spielberg,"['Thomas Keneally', 'Steven Zaillian']","['Liam Neeson', 'Ralph Fiennes', 'Ben Kingsley...",7,0,0


In [47]:
data['IMDb_Rating'].unique()

array([9.3, 9.2, 9. , 8.9, 8.8, 8.7, 8.6, 8.5, 8.4, 8.3, 8.2, 8.1, 8. ])

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213 entries, 0 to 212
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Movie_Name    213 non-null    object 
 1   Release_Yr    213 non-null    int64  
 2   Movie_Rating  205 non-null    object 
 3   Duration      213 non-null    object 
 4   IMDb_Rating   213 non-null    float64
 5   Popularity    213 non-null    object 
 6   Genre         213 non-null    object 
 7   Movie_Desc    213 non-null    object 
 8   Director      213 non-null    object 
 9   Writers       213 non-null    object 
 10  Movie_stars   212 non-null    object 
 11  Oscars_Won    213 non-null    int64  
 12  Awards_Won    213 non-null    int64  
 13  Nominations   213 non-null    int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 23.4+ KB


In [7]:
import sqlite3

conn = sqlite3.connect('movies.db')

In [17]:
create_table_query = """CREATE TABLE Movies_data (
    Movie_Name VARCHAR(255) NOT NULL,
    Release_Yr INT NOT NULL,
    Movie_Rating VARCHAR(255),
    Duration VARCHAR(255),
    IMDb_Rating FLOAT,
    Popularity VARCHAR(255),
    Genre VARCHAR(255),
    Movie_Desc TEXT,
    Director VARCHAR(255),
    Writers VARCHAR(255),
    Movie_stars VARCHAR(255),
    Oscars_Won INT,
    Awards_Won INT,
    Nominations INT
)"""

In [18]:
cursor = conn.cursor()
cursor.execute(create_table_query)

In [29]:
data.to_sql('Movies_data', conn, if_exists='append', index=False)

213

In [30]:
conn.commit()

In [39]:
cursor = conn.cursor()
cursor.execute("Select * from Movies_data")

In [ ]:
results = cursor.fetchall()
results

In [42]:


# Define the SELECT query
select_query = """
SELECT Movie_Name
FROM Movies_data
WHERE Awards_Won > 3
"""

# Execute the SELECT query
cursor.execute(select_query)

# Fetch all results
results = cursor.fetchall()

# Print the movie names
for row in results:
    print(row[0])




Inside Out


In [43]:

conn = sqlite3.connect('movies.db')
cursor = conn.cursor()

# Define the SQL table schema based on your DataFrame columns
table_schema = """
CREATE TABLE Movies (
    Movie_Name TEXT,
    Release_Yr INTEGER,
    Movie_Rating TEXT,
    Duration TEXT,
    IMDb_Rating REAL,
    Popularity INTEGER,
    Genre TEXT,
    Movie_Desc TEXT,
    Director TEXT,
    Writers TEXT,
    Movie_stars TEXT,
    Oscars_Won INTEGER,
    Awards_Won INTEGER,
    Nominations INTEGER
);
"""

# Execute the table creation query
cursor.execute(table_schema)

# Store the DataFrame in the SQLite database
data.to_sql('Movies', conn, if_exists='append', index=False)

# Commit changes
conn.commit()


In [48]:


# Define the SELECT query
select_query = """
SELECT Movie_Name
FROM Movies
WHERE IMDb_Rating > 8.9
"""

# Execute the SELECT query
cursor.execute(select_query)

# Fetch all results
results = cursor.fetchall()

# Print the movie names
for row in results:
    print(row[0])




The Shawshank Redemption
The Godfather
The Dark Knight
12 Angry Men
Schindler's List
The Lord of the Rings: The Return of the King
